In [1]:
import decimal

import pyarrow
import pymapd
import pandas as pd
import geopandas as gpd 
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon 

In [2]:
def cursor2df(cursor):
    has_geodata = {
        c.name: c.type_code in [13, 14, 15] for c in cursor.description
    }
    col_names = [c.name for c in cursor.description]

    df_class = gpd.GeoDataFrame if any(has_geodata.values()) else pd.DataFrame
    df = df_class(cursor.fetchall(), columns=col_names)

    for c, _has_geodata in has_geodata.items():
        if _has_geodata:
            df.loc[:, c] = df.loc[:, c].apply(shapely.wkt.loads)

    return df


In [3]:
con = pymapd.connect(
    host='localhost', 
    user='admin', 
    password='HyperInteractive',
    protocol='http', 
    port='6278'
)

In [4]:
con._client.get_version()

'5.2.2-20200518-7e9a996c75'

## Decimal field

In [5]:
table_name = 'mytabledecimal'

con.execute('''
DROP TABLE IF EXISTS {};
'''.format(table_name))

con.execute('''
CREATE TABLE {} (
    col_a DECIMAL(18, 15)
);
'''.format(table_name))

In [6]:
# df = pd.DataFrame({'a': pd.array([pd.array([1, 2], dtype='int'), pd.array([4, 5, 6, 7], dtype='int')])})
df = pd.DataFrame({
    'a': [decimal.Decimal('10.012345')],
})
df

,a
0,10.012345


In [7]:
df.dtypes

a    object
dtype: object

In [8]:
df.loc[:,['a']]

,a
0,10.012345


In [9]:
con.load_table(table_name, df, method='columnar')

In [10]:
cur = con.execute('select * from {}'.format(table_name))

In [11]:
cursor2df(cur)

,col_a
0,10.012345
